In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
from copy import deepcopy
from skimage import io
from tqdm.auto import tqdm
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torchvision
from torchvision import transforms as T
from torchvision.models import resnet50, vgg19
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from torchvision.models.detection import fasterrcnn_resnet50_fpn, MaskRCNN, FasterRCNN, retinanet_resnet50_fpn
from torchvision.models.detection.rpn import AnchorGenerator

In [ ]:
train_path = '../input/tensorflow-great-barrier-reef/train_images/'
df_path = '../input/tensorflow-great-barrier-reef/train.csv'
df = pd.read_csv(df_path).iloc[:, [0, 2, -1]]
test_df = pd.read_csv('../input/tensorflow-great-barrier-reef/test.csv').iloc[:, [0, 2]]
star_df = df[df['annotations'] != '[]']
non_df = df[df['annotations'] == '[]']
batch_size = 4
device = 'cuda' if torch.cuda.is_available() else 'cpu'

# first we pretrain the backbone for 5 epochs for classification to get better feature maps for detection.

In [ ]:
class ClassiData(Dataset):
    def __init__(self, train_path, star_df, non_df):
        self.star = star_df
        self.star = np.concatenate((self.star, np.ones((len(self.star),1))), axis = 1)
        self.non = non_df
        self.non = np.concatenate((self.non, np.zeros((len(self.non),1))), axis = 1)
        self.random_choice()
        self.path = train_path
        self.transforms = T.Compose([T.ToPILImage(), T.ToTensor(), T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
    def random_choice(self):
        rand = np.random.choice(len(self.non), len(self.star), replace = False)
        df = np.concatenate((self.star, self.non[rand]), axis = 0)
        np.random.shuffle(df)
        self.ids = df[:, 0]
        self.frames = df[:, 1]
        self.labels = df[:, -1]
    def __len__(self):
        return 2 * len(self.star)
    def __getitem__(self, idx):
        im_path = f'{self.path}video_{self.ids[idx]}/{self.frames[idx]}.jpg'
        label = self.labels[idx]
        image = io.imread(im_path)
        image = torch.from_numpy(image)
        image = image.permute((2, 0, 1))
        image = self.transforms(image)
        if np.random.rand() < 0.5:
            image = image.flip(-1)
        if idx == len(self):
            self.random_choice()
        return image, torch.tensor([label]).float()

In [ ]:
torch.cuda.empty_cache()

In [ ]:
resnet = torch.load('../input/resnet50/resnet50.pth')
backbone = resnet50()
backbone.load_state_dict(resnet)
backbone.avgpool = nn.AdaptiveAvgPool2d(7)
backbone.fc = nn.Sequential(nn.Linear(7*7*2048, 1024),
                            nn.ReLU(),
                            nn.Dropout(0.1),
                            nn.Linear(1024, 256),
                            nn.ReLU(),
                            nn.Dropout(0.1),
                            nn.Linear(256, 1),
                            nn.Sigmoid())

In [ ]:
train_data = ClassiData(train_path, star_df.values, non_df.values)
train_data = DataLoader(train_data, batch_size = batch_size)
optimizer = torch.optim.Adam(backbone.parameters(), lr = 0.0001)
criteria = nn.BCELoss()

In [ ]:
def train_model_classi(model, train_data, criteria, optimizer, epochs, val_data = None, device = device):
    min_val_loss = 10
    model_state = None
    model.to(device)
    for epoch in range(epochs):
        tr_loss = 0
        val_loss = 0
        train_step = 0
        acc = 0
        val_acc = 0
        for images, targets in tqdm(train_data):
            images = images.to(device)
            targets = targets.to(device)
            pred = model(images)
            loss = criteria(pred, targets)
            train_step += 1
            if train_step <= np.ceil(int(len(train_data)*0.8) / batch_size):
                acc += torch.sum(pred.detach().cpu() > 0.5).item()
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                tr_loss += loss.item()
            else:
                val_acc += torch.sum(pred.detach().cpu() > 0.5).item()
                val_loss += loss.item()
        
        tr_loss /= (int(len(train_data)*0.8))
        acc /= (int(len(train_data)*0.8))
        val_loss /= (len(train_data) - int(len(train_data)*0.8))
        val_acc /= (len(train_data) - int(len(train_data)*0.8))
        torch.cuda.empty_cache()
        if val_loss < min_val_loss:
            min_val_loss = val_loss
            model_state = deepcopy(model.state_dict())
        print(f'epoch: {epoch} loss: {tr_loss} acc: {acc} ;;;; val_loss: {val_loss} val_acc: {val_acc}')
    model.load_state_dict(model_state)

In [ ]:
train_model_classi(backbone, train_data, criteria, optimizer, 5)
torch.cuda.empty_cache()
batch_size = 1

# now we implement the dataset for object detection. this dataset returns the images and their respective target dictionary. target dictionaries consist of target bounding boxes, areas and labels.

In [ ]:
class StarfishData(Dataset):
    def __init__(self, train_path, df):
        np.random.shuffle(df)
        self.ids = df[:, 0]
        self.frames = df[:, 1]
        self.annotations = df[:, -1]
        self.path = train_path
        self.transforms = T.Compose([T.ToPILImage(), T.ToTensor()])
    def __len__(self):
        return len(self.ids)
    def __getitem__(self, idx):
        im_path = f'{self.path}video_{self.ids[idx]}/{self.frames[idx]}.jpg'
        annot = eval(self.annotations[idx])
        image = io.imread(im_path)
        boxes = [[x['x'], x['y'], x['x'] + x['width'], x['y'] + x['height']] for x in annot]
        areas = [x['width'] * x['height'] for x in annot]
        is_crowd = [0] * len(areas)
        labels = [1] * len(areas)
        image = torch.from_numpy(image)
        boxes = torch.as_tensor(boxes).type(torch.float32)
        areas = torch.as_tensor(areas).type(torch.float32)
        is_crowd = torch.as_tensor(is_crowd).type(torch.int64)
        labels = torch.as_tensor(labels).type(torch.int64)
        image = image.permute((2, 0, 1))
        image = self.transforms(image)
        if np.random.rand() < 0.5:
            image = image.flip(-1)
            boxes[:, [1, 3]] = image.size()[-1] - boxes[:, [3, 1]]
        target = {}
        target['boxes'] = boxes
        target['area'] = areas
        target['iscrowd'] = is_crowd
        target['labels'] = labels
        return image, target

In [ ]:
files = star_df.values
val_ind = np.random.choice(len(files), len(files) // 5, replace = False)
train_ind = np.delete(np.arange(len(files)), val_ind)
train_files = files[train_ind]
val_files = files[val_ind]

In [ ]:
train_data = StarfishData(train_path, np.concatenate((train_files, val_files), axis = 0))
train_data = DataLoader(train_data, batch_size = batch_size, collate_fn = lambda x:tuple(zip(*x)))
val_data = StarfishData(train_path, val_files)
val_data = DataLoader(val_data)

# here we read load the pretrained weights for the backbone.

In [ ]:
# vgg = torch.load('../input/vgg19/vgg19.pth')
# backbone = torchvision.models.vgg19()
# backbone.load_state_dict(vgg)
# backbone = backbone.features
# backbone.out_channels = 512
# anchor_generator = AnchorGenerator(sizes = ((64,128,256,512),) , aspect_ratios = ((0.5 , 1 , 2),))
# roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names = ['0'] , output_size = 7 , sampling_ratio = 2)
# model = FasterRCNN(backbone , rpn_anchor_generator = anchor_generator , box_roi_pool = roi_pooler , num_classes = 2)

In [ ]:
resnet = torch.load('../input/resnet50/resnet50.pth')
resnet = backbone.state_dict()
model = fasterrcnn_resnet50_fpn(pretrained_backbone = False, num_classes = 2)
# model.transform = torchvision.models.detection.transform.GeneralizedRCNNTransform(min_size = 100, max_size = 1300, fixed_size = (360, 640), image_mean = [0.485, 0.456, 0.406],image_std = [0.229, 0.224, 0.225])
state = model.backbone.body.state_dict()
for k in state.keys():
    if k in resnet.keys():
        state[k] = resnet[k]
model.backbone.body.load_state_dict(state)

<All keys matched successfully>

# finally it is time to train the model. we read images and targets from dataset and we calcuate the loss then we sum up all the losses and we back propagate and train the weights.

In [ ]:
def train_model(model, train_data, optimizer, epochs, val_data = None, device = device):
    min_val_loss = 10
    model_state = None
    model.to(device)
    for epoch in range(epochs):
        keys = None
        tr_loss = None
        val_loss = None
        train_step = 0
        for images, targets in tqdm(train_data):
            images = [image.to(device) for image in images]
            targets = [{k:v.to(device) for k , v in t.items()} for t in targets]
            loss_dict = model(images , targets)
            train_step += 1
            if train_step <= np.ceil(len(train_ind) / batch_size):
                loss = sum([x for x in loss_dict.values()])
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
                tr_loss = torch.stack(list(loss_dict.values())).detach().cpu() if tr_loss is None else tr_loss + torch.stack(list(loss_dict.values())).detach().cpu()
            else:
                val_loss = torch.stack(list(loss_dict.values())).detach().cpu() if val_loss is None else val_loss + torch.stack(list(loss_dict.values())).detach().cpu()
        
        keys = list(loss_dict.keys())
        tr_loss /= len(train_ind)
        train_str = [f'{keys[i]}:{tr_loss[i].item()}' for i in range(len(keys))]
        train_str = str.join('||' , train_str)
        val_loss /= len(val_ind)
        val_str = [f'{keys[i]}:{val_loss[i].item()}' for i in range(len(keys))]
        val_str = str.join('||' , val_str)
        torch.cuda.empty_cache()
        if torch.sum(val_loss) < min_val_loss:
            min_val_loss = torch.sum(val_loss)
            model_state = deepcopy(model.state_dict())
        print(f'{train_str};;;;{val_str}')
    model.load_state_dict(model_state)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0001)

In [ ]:
train_model(model, train_data, optimizer, 20)

  0%|          | 0/4919 [00:00<?, ?it/s]

loss_classifier:0.0926336869597435||loss_box_reg:0.01926822029054165||loss_objectness:0.5660269856452942||loss_rpn_box_reg:4.120506286621094;;;;loss_classifier:0.027312489226460457||loss_box_reg:0.006774798035621643||loss_objectness:0.5218324661254883||loss_rpn_box_reg:3.6710381507873535


  0%|          | 0/4919 [00:00<?, ?it/s]

loss_classifier:0.03353343531489372||loss_box_reg:0.014418660663068295||loss_objectness:0.5505380034446716||loss_rpn_box_reg:3.6624109745025635;;;;loss_classifier:0.05336063355207443||loss_box_reg:0.012831361964344978||loss_objectness:0.5959644913673401||loss_rpn_box_reg:3.7681069374084473


  0%|          | 0/4919 [00:00<?, ?it/s]

...

loss_classifier:0.05689128860831261||loss_box_reg:0.056882020086050034||loss_objectness:0.4784905016422272||loss_rpn_box_reg:2.1741628646850586;;;;loss_classifier:0.09095174819231033||loss_box_reg:0.07436792552471161||loss_objectness:0.44322827458381653||loss_rpn_box_reg:1.9288464784622192


  0%|          | 0/4919 [00:00<?, ?it/s]

loss_classifier:0.05351675674319267||loss_box_reg:0.055700644850730896||loss_objectness:0.4779527485370636||loss_rpn_box_reg:2.130586862564087;;;;loss_classifier:0.04211590439081192||loss_box_reg:0.05344346538186073||loss_objectness:0.44754552841186523||loss_rpn_box_reg:1.994939923286438


  0%|          | 0/4919 [00:00<?, ?it/s]

loss_classifier:0.06297570466995239||loss_box_reg:0.059919629245996475||loss_objectness:0.4694163203239441||loss_rpn_box_reg:2.147547960281372;;;;loss_classifier:0.05837774649262428||loss_box_reg:0.05128508433699608||loss_objectness:0.43657904863357544||loss_rpn_box_reg:2.0517847537994385


# in the end model is ready to be tested and in the following cells you can see how.

In [ ]:
class StarfishTest(Dataset):
    def __init__(self, train_path, df):
        self.path = train_path
        self.ids = df[:, 0]
        self.frames = df[:, 1]
        self.transforms = T.Compose([T.ToPILImage(), T.ToTensor()])
    def __len__(self):
        return len(self.ids)
    def __getitem__(self, idx):
        im_path = f'{self.path}video_{self.ids[idx]}/{self.frames[idx]}.jpg'
        image = io.imread(im_path)
        image = torch.from_numpy(image)
        image = image.permute((2, 0, 1))
        image = self.transforms(image)
        return image

In [ ]:
test_data = StarfishTest(train_path, test_df.values)
# test_data = DataLoader(test_data, collate_fn = lambda x:tuple(zip(*x)))

In [ ]:
def get_pred(pred):
    p = [f'{s} {b[0]} {b[1]} {b[2] - b[0]} {b[3] - b[1]}' for s,b,l in zip(pred['scores'],pred['boxes'],pred['labels']) if s > 0.5 and l == 1]
    return ' '.join(p)

In [ ]:
import greatbarrierreef
model.eval()
transform = T.Compose([T.ToPILImage(), T.ToTensor()])
env = greatbarrierreef.make_env()
iter_test = env.iter_test()
for (pixel_array, sample_prediction_df) in iter_test:
    im = transform(torch.from_numpy(pixel_array).permute((2, 0, 1))).to(device)
    sample_prediction_df['annotations'] = get_pred(model([im])[0])
    env.predict(sample_prediction_df)